## Point & Line

In [ ]:
import math
class Vector3:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def __str__(self):
        return "({:.2f}, {:.2f})".format(self.x, self.y)
    
        
    def distanceTo(self, point):
         return math.sqrt((self.x - point.x)**2 + (self.y - point.y)**2)
        
class Line:
    
    def __init__(self, A, B, C, x1=None,y1=None,x2=None,y2=None):
        self.A = A
        self.B = B
        self.C = C
        self.x1 = x1
        self.y1 = y1
        self.x2 = x2
        self.y2 = y2
        
    def __str__(self):
        sign = lambda x: "+" if x >= 0 else "-"
        return "{:.2f}x {} {:.2f}y {} {:.2f} = 0".format(self.A, sign(self.B),abs(self.B), sign(self.C), abs(self.C))
    
    @staticmethod
    def fromCoord(x1, y1, x2, y2):
        A = float(y1 - y2)
        B = float(x2 - x1)
        C = float(x1*y2 - x2*y1)
        return Line(A, B, C, x1, y1, x2, y2)
    
    def distanceToZero(self):
        x = abs(-self.C/self.A)
        y = abs(-self.C/self.B)
        hyp = (x**2 + y**2)**(0.5)
        return round(x*y/hyp, 2)
    
    
    def distanceToPoint(self, point):
        return round(abs((self.A*point.x + self.B*point.y + self.C)
                         /(self.A**2 +self.B**2)**(0.5)), 2)
    
    
    def isParallel(self, line):
        return abs(self.A*line.B - line.A*self.B)<= 0.001
            
    def intersection1(self, line):
        if self.isParallel(line):
            return False
        else:
            x = (line.C*self.B - self.C*line.B)/(self.A*line.B - self.B*line.A)
            y = (line.A*self.C - self.A*line.C)/(self.A*line.B - self.B*line.A)
            return x, y 
        
    def intersection(self, line):
        return "({:.2f}, {:.2f})".format(self.intersection1(line))
    
        
    def nearPoint(self, point):
        a1 = -self.B/self.A
        b1 = 1
        c1 = (self.B/self.A)*point.x - point.y
        return self.intersection1(Line(a1, b1, c1))
    
    
    def middlePoint(self, point):
        x, y = self.nearPoint(point)
        return (x/2, y/2)
        
    
    def sidePoint(self, point):
        side = self.A*point.x + self.B*point.y + self.C
        if abs(side) < 0.001:
             return 1
        elif side < 0:
             return 1
        else:
             return 0

    def oneSide(self, point1, point2):
        side1 = self.sidePoint(point1)
        side2 = self.sidePoint(point2)
        if (side1+side2)%2 == 0:
            return True
        else:
            return False
    
    
    def perpendicularLine(self, point):
        return Line(self.B/self.A, -1, -(self.B/self.A*point.x - point.y))
    
    
    def normalize(self):
        if self.C != 0:
            self.A, self.B, self.C = self.A/self.C, self.B/self.C, 1.00 
        else:
            if self.A != 0:
                self.A, self.B, self.C = 1.00, self.B/self.A, 0.00
            else:
                self.A, self.B, self.C = 0.00, 1.00, 0.00
        sign = lambda x: "+" if x >= 0 else "-"
        return "{:.2f}x {} {:.2f}y {} {:.2f} = 0".format(self.A, sign(self.B),abs(self.B), sign(self.C), abs(self.C))
    
    
    def parallelLine(self, point):
        return Line(self.A, self.B, -self.A*point.x - self.B*point.y).normalize()
    
    
    def projectionLength(self, point1, point2):
        return round(Point(*self.nearPoint(point1)).distanceTo(Point(*self.nearPoint(point2))),3)
    
    
    def middlePoint(self, point):
        x,y = self.nearPoint(point)
        return Point((point.x+x)/2, (point.y+y)/2)
    
    
    def symmetricPoint(self, point):
        prj = Point(*self.nearPoint(point))
        return Point(2*prj.x - point.x, 2*prj.y - point.y)
        
        
    def sign(p1,p2,p3):
         return (p1.x - p3.x) * (p2.y - p3.y) - (p2.x - p3.x) * (p1.y - p3.y)
    
    
    def insideTreug(self, point):
        p1 = Point(*self.intersection1(Line.fromCoord(0,0,1,0)))
        p2 = Point(*self.intersection1(Line.fromCoord(0,0,0,1)))
        p3 = Point(0,0)
        d1 = Line.sign(point, p1, p2)
        d2 = Line.sign(point, p2, p3)
        d3 = Line.sign(point, p3, p1)

        has_neg = (d1 < 0) or (d2 < 0) or (d3 < 0)
        has_pos = (d1 > 0) or (d2 > 0) or (d3 > 0)
        return not(has_neg and has_pos)
    
    def rotatedLine(self, point):
        x1, y1 = self.x1 - point.x, self.y1 - point.y
        x2, y2 = self.x2 - point.x, self.y2 - point.y
        rotated_x1, rotated_y1 = y1 + point.x, -x1 + point.y
        rotated_x2, rotated_y2 = y2 + point.x, -x2 + point.y
        line = self.fromCoord(rotated_x1, rotated_y1, rotated_x2, rotated_y2)
        line.normalize()
        return line
    
    def bisectrix(self, line2):
        if -self.A/self.B * -line2.A/line2.B == -1:
            return None
        
        k1 = math.sqrt(self.A**2 + self.B**2)
        k2 = math.sqrt(line2.A**2 + line2.B**2)
        if -self.A/self.B == -line2.A/line2.B:
            return Line(self.A/k1+line2.A/k2, self.B/k1+line2.B/k2, self.C/k1+line2.C/k2).normalize()
        
        b1 = Line(self.A/k1+line2.A/k2, self.B/k1+line2.B/k2, self.C/k1+line2.C/k2)
        b2 = Line(self.A/k1-line2.A/k2, self.B/k1-line2.B/k2, self.C/k1-line2.C/k2)
        ang1 = math.acos((b1.A*self.A + b1.B*self.B)/(math.sqrt(self.A**2 + self.B**2) * math.sqrt(b1.A**2 + b1.B**2)))
        ang2 = math.acos((b2.A*self.A + b2.B*self.B)/(math.sqrt(self.A**2 + self.B**2) * math.sqrt(b2.A**2 + b2.B**2)))
        if ang1 < ang2:
            return b1.normalize()
        else:
            return b2.normalize()
    


## Test

In [ ]:
line1 = Line.fromCoord(0,0,1,1)
line2 = Line.fromCoord(0,4,1,5)

line1.bisectrix(line2)


In [ ]:
line =  Line.fromCoord(0,1,1,0)
str(line)

In [ ]:
line =  Line.fromCoord(0,1,1,0)
str(line)
p1 = Point(5,5)
str(line.middlePoint(p1))

In [ ]:
line1 = Line.fromCoord(1, 0, 0, 1)
point1 = Point(3, 2)
line1.distanceToPoint(point1)

In [ ]:
line1 = Line.fromCoord(0, 1, 1, 0)
line2 = Line.fromCoord(5, 0, 0, 5)
line1.isParallel(line2)

In [ ]:
line1 = Line.fromCoord(0, 1, 1, 0)
line2 = Line.fromCoord(2, 3, 5, 8)
line1.intersection(line2)

In [ ]:
line1 = Line.fromCoord(0, 1, 1, 0)
point1 = Point(3, 4)
line1.nearPoint(point1)

In [ ]:
line1 = Line.fromCoord(2, 0, 0, 2)
str(line1)

In [ ]:
line = Line.fromCoord(7.25,0,-12.35,0)
line.normalize()